In [111]:
import fasttext
import os
import umap
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.cluster import KMeans
from matplotlib.lines import Line2D
from scipy.spatial.distance import cdist, cosine, euclidean
import tensorflow_hub as hub
from sklearn.decomposition import PCA, FastICA
from scipy.stats import ttest_ind, ttest_1samp
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans

In [112]:
DATA_FPATH = '../data/reddit/'
MODEL_FPATH = '../models/reddit/'
USE_PATH = "/Users/harinisuresh/Dev/tf_hub/" + 'universal-sentence-encoder_4/' # replace with your path
subreddits = ['askscience', 'conspiracy', 'funny', 'hillaryclinton', 'history']

In [113]:
all_comments = pd.read_csv(DATA_FPATH + 'all_comments_df')
embed = hub.load(USE_PATH);
all_comments.head()

Unnamed: 0                                               body  \
0           0  You can tell this is fake because it shows a b...   
1           1  They're purple because she's dead.\n\nLol sorr...   
2           2                           The fat lady is singing.   
3           3  Good thing volks never break down. XD Shitty k...   
4           5              I'm dreaming of a bright christmas.\n   

        subreddit  moderated  
0           funny          1  
1  hillaryclinton          1  
2           funny          1  
3           funny          1  
4           funny          1

In [114]:
sub = 'funny'
projection_df = pd.read_csv(DATA_FPATH + sub + '/projection_df')

In [115]:
embeddings = np.load(DATA_FPATH + sub + '/embs.npy')

In [116]:
projection_df

comment_id                                            comment  label  \
0               0  You can tell this is fake because it shows a b...      1   
1               1                           The fat lady is singing.      1   
2               2  Good thing volks never break down. XD Shitty k...      1   
3               3              I'm dreaming of a bright christmas.\n      1   
4               4  I'd be sad too if my mom got knocked up in hig...      1   
...           ...                                                ...    ...   
34342       34342  I feel pretty stupid right now.  I'm deleting ...      0   
34343       34343                           That's amazing thank you      0   
34344       34344                                 love the honesty\n      0   
34345       34345  At least an effort was made to obey the rules....      0   
34346       34346  Why?\n\nYou're young and can keep up with them...      0   

        pred    umap_x    umap_y     pca_x     pca_y     tsne_x     tsne_y  
0      0.988 -3.045098  0.085387 -0.143643  0.085669  55.842620  13.659914  
1      0.997  1.575318 -1.997435 -0.008068  0.004616   9.396464  18.984072  
2      0.990 -0.131999  1.414051 -0.106509  0.059272  26.409739 -35.093937  
3      0.995  0.491807 -4.564635  0.071565  0.069409 -11.741515  32.897507  
4      0.961 -2.683635 -1.535356 -0.131564  0.022218  13.745846  -3.450827  
...      ...       ...       ...       ...       ...        ...        ...  
34342  0.141  1.146495 -4.195731 -0.075867  0.301665  10.959727  30.994236  
34343  0.011  0.889189 -5.888040  0.098362  0.168887 -28.783457  42.777153  
34344  0.228 -0.402147 -4.890916  0.167125  0.284558 -25.468122  20.661785  
34345  0.031 -1.510421  0.605365 -0.097116 -0.142629  46.181120 -12.363643  
34346  0.187 -1.226840 -0.120017 -0.081217 -0.292853  26.715643  -8.299600  

[34347 rows x 10 columns]

In [126]:
np.save(
    DATA_FPATH + sub + '/embs.npy', embed(projection_df.drop_duplicates('comment').comment).numpy()
)

In [128]:
projection_df.drop_duplicates('comment').to_csv(DATA_FPATH + sub + '/projection_df')

In [154]:
projection_df = pd.read_csv(DATA_FPATH + sub + '/projection_df')

In [166]:
projection_df['comment'] = projection_df.comment.apply(lambda x: x.strip())

In [168]:
projection_df = projection_df.drop_duplicates('comment')

In [169]:
projection_df = projection_df[['comment_id', 'comment', 'label', 'pred', 'umap_x', 'umap_y', 'pca_x', 'pca_y', 'tsne_x', 'tsne_y']]
projection_df = projection_df.reset_index(drop=True)
projection_df['comment_id'] = np.arange(0, projection_df.shape[0])

In [171]:
projection_df.to_csv(DATA_FPATH + sub + '/projection_df')

In [172]:
np.save(
    DATA_FPATH + sub + '/embs.npy', embed(projection_df.comment).numpy()
)

In [14]:
def getSimilarExamples():
    allExampleIds = json.loads(request.args.get('allExamples'))
    allEmbeddings = embeddings
    embeddingMean = np.mean(allEmbeddings, axis=0)
    filters = request.args.get('filters')
    if filters is not None:
        filters = json.loads(filters)
        print(filters)
        probRange = json.loads(filters['probRange'])
        queryString = json.loads(filters['queryString'])
        labelFilter = json.loads(filters['labelFilter'])
        labelFilter = np.where(np.array(labelFilter) == True)[0]
        print(probRange)
        print(allExamples)
        similarExamples, modelPreds, labels, clusters = getKNNFromVector(embeddingMean, allExamples, probRange, labelFilter, queryString)
    else:
        similarExamples, modelPreds, labels, clusters = getKNNFromVector(embeddingMean, allExamples)
    
    print(similarExamples)
    return json.jsonify(similarExamples=similarExamples, modelPreds=modelPreds, labels=labels, clusters=clusters)

def getKNNFromVector(vec, allExamples, probRange=[0,1], labelFilter=[0,1], queryString="", n=30):
    print(queryString)
    
    nonDuplicateIdx = [c not in allExamples for c in comments]
    print('here, non dup idx')
    print(np.where(np.array(nonDuplicateIdx)==False))
    print(np.array(comments)[np.where(np.array(nonDuplicateIdx)==False)])

    stringQueryIdx = [True if re.match(r'.*\b%s\b.*' % queryString, c, re.IGNORECASE) else False for c in comments]

    filteredIdx = np.where(
        (preds >= probRange[0]) & (preds <= probRange[1]) &
        np.isin(labels, labelFilter) & stringQueryIdx & nonDuplicateIdx
    )

    print(np.isin(labels, labelFilter))
    # print(stringQueryIdx)

    filteredEmbeddings = embeddings[filteredIdx]
    dist_vec = cdist([vec], filteredEmbeddings, 'cosine')
    top_vec_idx = np.argsort(dist_vec[0])[1:n+1]
    print(top_vec_idx)
    
    top_vecs = filteredEmbeddings[top_vec_idx]
    kmeans = KMeans(n_clusters=3, random_state=0).fit(top_vecs)
    clusters = [int(c) for c in kmeans.labels_]

    top_comment_ids = filteredIdx[top_vec_idx]
    
    top_comments = np.array(comments)[filteredIdx][top_vec_idx]
    top_comments = [c for c in top_comments]
    
    top_labels = np.array(labels)[filteredIdx][top_vec_idx]
    top_labels = [int(l) for l in top_labels]

    modelPreds = getModelPredsHelper(top_comments)

    return top_comment_ids, modelPreds, top_labels, clusters


In [178]:
projection_df[projection_df.comment == "shut up"]

comment_id  comment  label  pred    umap_x    umap_y     pca_x  \
2664        2664  shut up      1   1.0 -2.664235 -3.238206  0.384865   

         pca_y    tsne_x    tsne_y  
2664  0.179599 -25.76352  6.855419

In [105]:
x = np.array([1,2,3,4,5,6])

In [106]:
x[x < 4]

array([1, 2, 3])

In [110]:
projection_df.comment_id


0            0
1            1
2            2
3            3
4            4
         ...  
34342    34342
34343    34343
34344    34344
34345    34345
34346    34346
Name: comment_id, Length: 34347, dtype: int64